<a href="https://colab.research.google.com/github/rho-selynn/592-final-project-YOLO/blob/main/592_final_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import dependencies and libraries

In [1]:
import torch
from IPython.display import Image
import os

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!git clone https://github.com/ultralytics/yolov5

# Change the current working directory.
%cd yolov5 
!pip install -qr requirements.txt # install dependencies from requirements to use Yolov5 with quiet option

Cloning into 'yolov5'...
remote: Enumerating objects: 13049, done.
remote: Total 13049 (delta 0), reused 0 (delta 0), pack-reused 13049
Receiving objects: 100% (13049/13049), 11.97 MiB | 26.14 MiB/s, done.
Resolving deltas: 100% (9070/9070), done.
/content/yolov5
     |████████████████████████████████| 596 kB 29.7 MB/s 


In [4]:
!pip install tensorflow==2.3.1 
!pip install tensorboard==2.4.1

     |████████████████████████████████| 320.4 MB 25 kB/s 
     |████████████████████████████████| 20.1 MB 1.3 MB/s 
     |████████████████████████████████| 459 kB 55.2 MB/s 
     |████████████████████████████████| 2.9 MB 48.5 MB/s 
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.6
    Uninstalling numpy-1.21.6:
      Successfully uninstalled numpy-1.21.6
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.8.0
    Uninstalling tensorflow-estimator-2.8.0:
      Successfully uninstalled tensorflow-estimator-2.8.0
  Attempting uninstall: h5py
    Found existing installation: h5py 3.1.0
    Uninstalling h5py-3.1.0:
      Successfully uninstalled h5py-3.1.0
  Attempting uninstall: gast
    Found existing installation: gast 0.5.3
    Uninstalling gast-0.5.3:
      Successfully uninstalled gast-0.5.3
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.8.0
    Uninstalling tensorflow-2.8.0:
 

     |████████████████████████████████| 10.6 MB 17.7 MB/s 
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.8.0
    Uninstalling tensorboard-2.8.0:
      Successfully uninstalled tensorboard-2.8.0


Downloading the dataset

In [5]:
os.environ['KAGGLE_CONFIG_DIR'] = "/content/drive/My Drive/ME 592/Final Project/Kaggle"

In [8]:
cd "/content/drive/My Drive/ME 592/Final Project/Yolov5/data"

/content/drive/My Drive/ME 592/Final Project/Yolov5/data


In [7]:
!kaggle datasets download -d valentynsichkar/traffic-signs-dataset-in-yolo-format

 96% 241M/250M [00:01<00:00, 172MB/s]
100% 250M/250M [00:01<00:00, 136MB/s]


In [9]:
!unzip traffic-signs-dataset-in-yolo-format.zip

Archive:  traffic-signs-dataset-in-yolo-format.zip
  inflating: classes.names           
  inflating: getting-full-path.py    
  inflating: test.txt                
  inflating: traffic-sign-to-test.mp4  
  inflating: train.txt               
  inflating: ts/ts/00000.jpg         
  inflating: ts/ts/00000.txt         
  inflating: ts/ts/00001.jpg         
  inflating: ts/ts/00001.txt         
  inflating: ts/ts/00002.jpg         
  inflating: ts/ts/00002.txt         
  inflating: ts/ts/00003.jpg         
  inflating: ts/ts/00003.txt         
  inflating: ts/ts/00004.jpg         
  inflating: ts/ts/00004.txt         
  inflating: ts/ts/00005.jpg         
  inflating: ts/ts/00005.txt         
  inflating: ts/ts/00006.jpg         
  inflating: ts/ts/00006.txt         
  inflating: ts/ts/00007.jpg         
  inflating: ts/ts/00007.txt         
  inflating: ts/ts/00008.jpg         
  inflating: ts/ts/00008.txt         
  inflating: ts/ts/00009.jpg         
  inflating: ts/ts/00009.txt       

In [10]:
ls

classes.names                             train.txt
getting-full-path.py                      ts/
test.txt                                  ts_data.data
traffic-signs-dataset-in-yolo-format.zip  yolov3_ts_test.cfg
traffic-sign-to-test.mp4                  yolov3_ts_train.cfg


Divide dataset into "train" and "val" folders

In [15]:
import os
from random import choice
import shutil

#arrays to store file names
imgs =[]
xmls =[]
# /content/drive/My Drive/ME 592/Final Project/Kaggle
#setup dir names
trainPath = '/content/drive/My Drive/ME 592/Final Project/Yolov5/dataset/images/train'
valPath = '/content/drive/My Drive/ME 592/Final Project/Yolov5/dataset/images/val'
crsPath = '/content/drive/My Drive/ME 592/Final Project/Yolov5/data/ts/ts' #dir where images and annotations stored

#setup ratio (val ratio = rest of the files in origin dir after splitting into train and test)
train_ratio = 0.8
val_ratio = 0.2


#total count of imgs
totalImgCount = len(os.listdir(crsPath))/2

#soring files to corresponding arrays
for (dirname, dirs, files) in os.walk(crsPath):
    for filename in files:
        if filename.endswith('.txt'):
            xmls.append(filename)
        else:
            imgs.append(filename)


#counting range for cycles
countForTrain = int(len(imgs)*train_ratio)
countForVal = int(len(imgs)*val_ratio)
print("training images are : ",countForTrain)
print("Validation images are : ",countForVal)

training images are :  592
Validation images are :  148


In [16]:
trainimagePath = '/content/drive/My Drive/ME 592/Final Project/Yolov5/dataset/images/train'
trainlabelPath = '/content/drive/My Drive/ME 592/Final Project/Yolov5/dataset/labels/train'
valimagePath = '/content/drive/My Drive/ME 592/Final Project/Yolov5/dataset/images/val'
vallabelPath = '/content/drive/My Drive/ME 592/Final Project/Yolov5/dataset/labels/val'
#cycle for train dir
for x in range(countForTrain):

    fileJpg = choice(imgs) # get name of random image from origin dir
    fileXml = fileJpg[:-4] +'.txt' # get name of corresponding annotation file

    #move both files into train dir
    #shutil.move(os.path.join(crsPath, fileJpg), os.path.join(trainimagePath, fileJpg))
    #shutil.move(os.path.join(crsPath, fileXml), os.path.join(trainlabelPath, fileXml))
    shutil.copy(os.path.join(crsPath, fileJpg), os.path.join(trainimagePath, fileJpg))
    shutil.copy(os.path.join(crsPath, fileXml), os.path.join(trainlabelPath, fileXml))


    #remove files from arrays
    imgs.remove(fileJpg)
    xmls.remove(fileXml)



#cycle for test dir   
for x in range(countForVal):

    fileJpg = choice(imgs) # get name of random image from origin dir
    fileXml = fileJpg[:-4] +'.txt' # get name of corresponding annotation file

    #move both files into train dir
    #shutil.move(os.path.join(crsPath, fileJpg), os.path.join(valimagePath, fileJpg))
    #shutil.move(os.path.join(crsPath, fileXml), os.path.join(vallabelPath, fileXml))
    shutil.copy(os.path.join(crsPath, fileJpg), os.path.join(valimagePath, fileJpg))
    shutil.copy(os.path.join(crsPath, fileXml), os.path.join(vallabelPath, fileXml))
    
    #remove files from arrays
    imgs.remove(fileJpg)
    xmls.remove(fileXml)

#rest of files will be validation files, so rename origin dir to val dir
#os.rename(crsPath, valPath)
shutil.move(crsPath, valPath)

'/content/drive/My Drive/ME 592/Final Project/Yolov5/dataset/images/val/ts'

Create dataset config file

In [22]:
# I will write the contents of the cell to a file
%%writefile /content/yolov5/data/traffic_sign_data.yaml 

# Train/val/test sets as 1) dir: path/to/imgs, 2) file: path/to/imgs.txt, or 3) list: [path/to/imgs1, path/to/imgs2, ..]
path: "/content/drive/My Drive/ME 592/Final Project/Yolov5/dataset/images/" # dataset root dir
train: images/train/ # train images (relative to 'path')
val: images/val/ # val images (relative to 'path')
test:  # test images (optional)

# Classes
nc: 4  # number of classes

# class names
names: ['speed limit', 'yield', 'mandatory', 'other']

Overwriting /content/yolov5/data/traffic_sign_data.yaml


Training

In [23]:
pwd

'/content/yolov5'

In [24]:
cd /content/yolov5

/content/yolov5


In [25]:
# Training YOLOv5s on traffic sign dataset for 30 epochs
!python train.py --img 1360 --batch 16 --epochs 30 --data traffic_sign_data.yaml --weights yolov5s.pt --cache

train: weights=yolov5s.pt, cfg=, data=traffic_sign_data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=30, batch_size=16, imgsz=1360, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v6.1-165-g1a3ecb8 torch 1.11.0+cu113 CPU

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0.0, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, degrees=0.0, translate=0.1, scale=0